In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn

In [3]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [4]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [5]:
train.columns = ['passenger_id', 'home_planet', 'cryo_sleep', 'cabin', 'destination', 'age', 'vip', 'room_service', 'food_court', 'shopping_mall', 'spa', 'vr_deck', 'name', 'transported']
test.columns = ['PassengerId', 'home_planet', 'cryo_sleep', 'cabin', 'destination', 'age', 'vip', 'room_service', 'food_court', 'shopping_mall', 'spa', 'vr_deck', 'name']

In [6]:
train.columns

Index(['passenger_id', 'home_planet', 'cryo_sleep', 'cabin', 'destination',
       'age', 'vip', 'room_service', 'food_court', 'shopping_mall', 'spa',
       'vr_deck', 'name', 'transported'],
      dtype='object')

In [7]:
train.dtypes

passenger_id      object
home_planet       object
cryo_sleep        object
cabin             object
destination       object
age              float64
vip               object
room_service     float64
food_court       float64
shopping_mall    float64
spa              float64
vr_deck          float64
name              object
transported         bool
dtype: object

In [8]:
train['home_planet'].unique()

array(['Europa', 'Earth', 'Mars', nan], dtype=object)

In [9]:
test['home_planet'].unique()

array(['Earth', 'Europa', 'Mars', nan], dtype=object)

In [10]:
train['home_planet'].dropna(inplace = True)
train[train['home_planet'] == 'Europa'] = 0
train[train['home_planet'] == 'Earth'] = 1
train[train['home_planet'] == 'Mars'] = 2

test['home_planet'].fillna(1, inplace = True)
test[test['home_planet'] == 'Europa'] = 0
test[test['home_planet'] == 'Earth'] = 1
test[test['home_planet'] == 'Mars'] = 2

In [11]:
train['home_planet'].value_counts()

1    4602
0    2131
2    1759
Name: home_planet, dtype: int64

In [12]:
test['home_planet'].value_counts()

1    2350
0    1002
2     925
Name: home_planet, dtype: int64

In [13]:
train['cryo_sleep'].isnull().unique()

array([False,  True])

In [14]:
test['cryo_sleep'].isnull().unique()

array([False,  True])

In [15]:
train['cryo_sleep'].unique()

array([0, 1, 2, nan], dtype=object)

In [16]:
test['cryo_sleep'].unique()

array([1, 0, 2, nan], dtype=object)

In [17]:
train['cryo_sleep'].value_counts()

1    4677
0    2255
2    1759
Name: cryo_sleep, dtype: int64

In [18]:
test['cryo_sleep'].value_counts()

1    2291
0    1058
2     925
Name: cryo_sleep, dtype: int64

In [19]:
test['cryo_sleep'].fillna(1, inplace = True)

In [20]:
test[test['cryo_sleep'] == False] = 0

In [21]:
test['cryo_sleep'].unique()

array([1, 0, 2], dtype=object)

In [22]:
train['cabin'].unique()

array([0, 1, 2, 'E/3/S', 'A/0/P', 'D/6/P', 'F/46/S', 'C/11/S', 'G/41/S',
       'G/42/S', 'F/61/S', 'G/64/P', 'D/15/S', 'F/99/P', 'E/25/S',
       'F/86/S', 'B/19/S', 'F/92/S', 'C/18/P', 'F/95/S', 'F/103/S',
       'E/39/S', 'F/141/P', 'E/56/S', 'A/9/S', 'F/189/P', 'G/151/S',
       'G/152/P', 'F/218/P', 'B/38/P', 'T/0/P', 'D/41/P', 'F/271/P',
       'G/213/P', 'G/223/S', 'E/111/S', nan, 'E/125/S', 'F/357/P',
       'G/292/S', 'G/305/S', 'G/311/S', 'G/322/S', 'G/330/P', 'G/327/S',
       'G/369/S', 'G/373/P', 'G/386/P', 'C/86/S', 'D/72/P', 'G/395/S',
       'G/396/S', 'E/149/P', 'A/19/P', 'F/500/S', 'E/159/P', 'G/430/P',
       'F/557/P', 'G/433/S', 'G/447/S', 'G/455/P', 'F/584/P', 'G/450/S',
       'F/553/S', 'E/179/P', 'F/570/S', 'F/629/P', 'G/492/S', 'F/659/P',
       'G/508/S', 'F/631/S', 'D/108/S', 'F/703/P', 'B/130/S', 'G/578/P',
       'F/664/S', 'G/574/S', 'G/594/P', 'F/768/P', 'E/222/P', 'G/620/P',
       'G/626/S', 'F/797/P', 'C/140/S', 'F/736/S', 'G/661/S', 'G/667/P',
      

In [23]:
del train['cabin']

In [24]:
test['cabin'].value_counts()

1           2263
0           1058
2            925
G/1399/S       1
C/322/S        1
F/1730/P       1
G/1255/P       1
C/280/S        1
F/1441/S       1
G/747/S        1
A/83/S         1
E/415/S        1
F/1256/P       1
F/1207/P       1
F/947/S        1
G/771/P        1
C/177/S        1
E/471/S        1
G/731/P        1
E/286/S        1
F/832/S        1
F/798/S        1
G/587/P        1
E/228/S        1
C/124/S        1
D/87/P         1
G/430/S        1
G/400/S        1
A/17/P         1
G/332/P        1
G/226/S        1
C/44/S         1
G/157/P        1
D/275/P        1
Name: cabin, dtype: int64

In [25]:
del test['cabin']

In [26]:
train.columns

Index(['passenger_id', 'home_planet', 'cryo_sleep', 'destination', 'age',
       'vip', 'room_service', 'food_court', 'shopping_mall', 'spa', 'vr_deck',
       'name', 'transported'],
      dtype='object')

In [27]:
train['destination'].unique()

array([0, 1, 2, 'TRAPPIST-1e', '55 Cancri e', 'PSO J318.5-22', nan],
      dtype=object)

In [28]:
test['destination'].unique()

array([1, 0, 2, 'TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e'],
      dtype=object)

In [29]:
train['destination'].dropna(inplace = True)
train[train['destination'] == 'TRAPPIST-1e'] = 3
train[train['destination'] == '55 Cancri e'] = 4
train[train['destination'] == 'PSO J318.5-22'] = 5

In [30]:
test['destination'].fillna(1, inplace = True)
test[test['destination'] == 'TRAPPIST-1e'] = 3
test[test['destination'] == '55 Cancri e'] = 4
test[test['destination'] == 'PSO J318.5-22'] = 5

In [31]:
train['destination'].value_counts()

1    4602
0    2131
2    1759
3     150
4      31
5      16
Name: destination, dtype: int64

In [32]:
test['destination'].value_counts()

1    2263
0    1058
2     925
3      18
4       9
5       4
Name: destination, dtype: int64

In [33]:
train['age'].unique()

array([ 0.,  1.,  2.,  3.,  4.,  5., 23., 43., 52., 12.])

In [34]:
train['vip'].unique()

array([0, 1, 2, 3, 4, 5, nan], dtype=object)

In [35]:
test['vip'].unique()

array([1, 0, 2, 3, 5, 4], dtype=object)

In [36]:
train['vip'].dropna(inplace = True)

In [37]:
train['vip'].value_counts()

1    4602
0    2134
2    1759
3     150
4      31
5      16
Name: vip, dtype: int64

In [38]:
test['vip'].value_counts()

1    2263
0    1058
2     925
3      18
4       9
5       4
Name: vip, dtype: int64

In [39]:
train['room_service'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5., 348.])

In [40]:
train['food_court'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5., 645.])

In [41]:
train['shopping_mall'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5., 213.])

In [42]:
train['spa'].unique()

array([0., 1., 2., 3., 4., 5., 7.])

In [43]:
train['name'].unique()

array([0, 1, 2, 3, 4, 5, 'Blanie Floydendley', 'Winia Blanglison',
       'Bobara Alshipson', 'Walles Bakerrison'], dtype=object)

In [44]:
del train['name']

In [45]:
test['name'].unique()

array([1, 0, 2, 3, 5, 4], dtype=object)

In [46]:
del test['name']

In [47]:
train.columns

Index(['passenger_id', 'home_planet', 'cryo_sleep', 'destination', 'age',
       'vip', 'room_service', 'food_court', 'shopping_mall', 'spa', 'vr_deck',
       'transported'],
      dtype='object')

In [48]:
train['vr_deck'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5., 368., 701., 395.])

In [49]:
train['transported'].unique()

array([0, 1, 2, 3, 4, 5], dtype=object)

In [50]:
train['transported'].value_counts()

1    4604
0    2133
2    1759
3     150
4      31
5      16
Name: transported, dtype: int64

In [51]:
train.dtypes

passenger_id      object
home_planet       object
cryo_sleep        object
destination       object
age              float64
vip               object
room_service     float64
food_court       float64
shopping_mall    float64
spa              float64
vr_deck          float64
transported       object
dtype: object

In [52]:
train.dropna(axis = 0, inplace = True)

train['home_planet'] = train['home_planet'].astype(str).astype(int)
train['cryo_sleep'] = train['cryo_sleep'].astype(str).astype(int)
train['destination'] = train['destination'].astype(str).astype(int)
train['vip'] = train['vip'].astype(str).astype(int)
train['transported'] = train['transported'].astype(str).astype(int)

test['home_planet'] = test['home_planet'].astype(str).astype(int)
test['cryo_sleep'] = test['cryo_sleep'].astype(str).astype(int)
test['destination'] = test['destination'].astype(str).astype(int)
test['vip'] = test['vip'].astype(str).astype(int)

In [53]:
train.dtypes

passenger_id      object
home_planet        int64
cryo_sleep         int64
destination        int64
age              float64
vip                int64
room_service     float64
food_court       float64
shopping_mall    float64
spa              float64
vr_deck          float64
transported        int64
dtype: object

In [54]:
X = train.drop('transported', axis = 1)

y = train['transported'].astype(str).astype(int)

X = train.drop('passenger_id', axis = 1)

y = train['passenger_id'].astype(str).astype(int)

train_x, val_x, train_y, val_y = train_test_split(X, y)

In [55]:
X.dtypes

home_planet        int64
cryo_sleep         int64
destination        int64
age              float64
vip                int64
room_service     float64
food_court       float64
shopping_mall    float64
spa              float64
vr_deck          float64
transported        int64
dtype: object

In [56]:
X['destination'].unique()

array([0, 1, 2, 3, 4, 5])

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

max_accuracy = 0
mn = 0
md = 0

for n in range(1, 11):
    for d in range(1, 11):        
        model = RandomForestClassifier(n_estimators = n, max_depth = d, random_state = 1)
        model.fit(train_x, train_y)
        predictions = model.predict(val_x)
        
        accuracy = 100 - (100 * (mean_absolute_error(predictions, val_y)))
        
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            mn = n
            md = d

print(max_accuracy)
print(mn)
print(md)

100.0
1
5


In [58]:
model = RandomForestClassifier(n_estimators = mn, max_depth = md, random_state = 1)
model.fit(train_x, train_y)

predictions = model.predict(test)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index = False)
print('Your submission was successfully saved!')

Your submission was successfully saved!


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- PassengerId
Feature names seen at fit time, yet now missing:
- transported

  warnings.warn(message, FutureWarning)
